In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [5]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 1us/step


In [6]:
# 让 numpy 数据更易读。
np.set_printoptions(precision=3, suppress=True)

In [7]:
# 打印 CSV 文件的前几行
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [12]:
# 指定包含模型需要预测的值的列。
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

# 现在从文件中读取 CSV 数据并且创建 dataset。
def get_dataset(file_path):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12, # 为了示例更容易展示，手动设置较小的值
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


### dataset 中的每个条目都是一个批次，用一个元组（多个样本，多个标签）表示。样本中的数据组织形式是以列为主的张量（而不是以行为主的张量），每条数据中包含的元素个数就是批次大小（batch_size这个示例中是 12）。

In [14]:
examples, labels = next(iter(raw_train_data)) # 第一个批次
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

EXAMPLES: 
 OrderedDict([('sex', <tf.Tensor: id=176, shape=(12,), dtype=string, numpy=
array([b'female', b'female', b'male', b'male', b'female', b'male',
       b'male', b'male', b'male', b'male', b'female', b'male'],
      dtype=object)>), ('age', <tf.Tensor: id=168, shape=(12,), dtype=float32, numpy=
array([ 4., 28., 48., 60., 30., 18., 27., 41.,  3., 26., 28., 24.],
      dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: id=174, shape=(12,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0], dtype=int32)>), ('parch', <tf.Tensor: id=175, shape=(12,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1], dtype=int32)>), ('fare', <tf.Tensor: id=173, shape=(12,), dtype=float32, numpy=
array([ 23.   ,   7.896,   7.854,  26.55 ,   8.663,   7.796,  26.   ,
        14.108,  26.   ,   7.775,   7.75 , 247.521], dtype=float32)>), ('class', <tf.Tensor: id=170, shape=(12,), dtype=string, numpy=
array([b'Second', b'Third', b'Third', b'First', b'Third', b'Third',
       b'S

# 数据预处理
## 分类数据

### CSV 数据中的有些列是分类的列。也就是说，这些列只能在有限的集合中取值。

### 使用 tf.feature_column API 创建一个 tf.feature_column.indicator_column 集合，每个 tf.feature_column.indicator_column 对应一个分类的列。

In [16]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))
    
categorical_columns
# 这将是后续构建模型时处理输入数据的一部分。



[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

## 连续数据

### 连续数据需要标准化。

### 写一个函数标准化这些值，然后将这些值改造成 2 维的张量。

In [20]:
def process_continuous_data(mean, data):
    # 标准化数据
    data = tf.cast(data, tf.float32) * 1/(2*mean)
    return tf.reshape(data, [-1, 1])

MEANS = {
    'age' : 29.631308,
    'n_siblings_spouses' : 0.545455,
    'parch' : 0.379585,
    'fare' : 34.385399
}

numerical_columns = []
import functools

for feature in MEANS.keys():
    num_col = tf.feature_column.numeric_column(feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
    numerical_columns.append(num_col)
numerical_columns

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x63bca9200>, 29.631308)),
 NumericColumn(key='n_siblings_spouses', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x63bca9200>, 0.545455)),
 NumericColumn(key='parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x63bca9200>, 0.379585)),
 NumericColumn(key='fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x63bca9200>, 34.385399))]

## 创建预处理层

### 将这两个特征列的集合相加，并且传给 tf.keras.layers.DenseFeatures 从而创建一个进行预处理的输入层。

In [21]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numerical_columns)

## 构建模型
### 从 preprocessing_layer 开始构建 tf.keras.Sequential。

In [22]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [23]:
train_data = raw_train_data.shuffle(500)
test_data = raw_test_data

In [24]:
model.fit(train_data, epochs=20)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Epoch 1/20
53/53 [==============================] - 4s 82ms/step - loss: 0.5428 - accuracy: 0.7640.5579 - 
Epoch 2/20
53/53 [==============================] - 1s 12ms/step - loss: 0.4395 - accuracy: 0.8086
Epoch 3/20
53/53 [==============================] - 1s 13ms/step - loss: 0.4278 - accuracy: 0.8134
Epoch 4/20
53/53 [==============================] - 1s 13ms/step - loss: 0.4090 - accuracy: 0.8182
Epoch 5/20
53/53 [==============================] - 1s 12ms/step - loss: 0.3966 - accuracy: 0.8309
Epoch 6/20
53/53 [==============================] - 1s 13ms/step - loss: 0.3947 - accuracy: 0.8325
Epoch 7/20
53/53 [==============================] - 1s 13ms/step - loss: 0.3817 - accuracy: 0.8373
Epoch 8/20
53/53 [==============================] -

In [25]:
model.evaluate(test_data)

22/22 [==============================] - 1s 60ms/step - loss: 0.4498 - accuracy: 0.8030


[0.4498139883984219, 0.8030303]

## 使用 tf.keras.Model.predict 推断一个批次或多个批次的标签。

In [27]:
predictions = model.predict(test_data)

# 显示部分结果
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))



Predicted survival: 48.96%  | Actual outcome:  SURVIVED
Predicted survival: 40.46%  | Actual outcome:  DIED
Predicted survival: 11.38%  | Actual outcome:  DIED
Predicted survival: 0.84%  | Actual outcome:  SURVIVED
Predicted survival: 22.29%  | Actual outcome:  DIED
Predicted survival: 12.56%  | Actual outcome:  DIED
Predicted survival: 86.87%  | Actual outcome:  DIED
Predicted survival: 3.01%  | Actual outcome:  DIED
Predicted survival: 70.47%  | Actual outcome:  DIED
Predicted survival: 11.44%  | Actual outcome:  SURVIVED
